In [10]:
import ijson
import torch
import numpy as np
import pandas as pd
import pickle
import sys
from tqdm import tqdm
from sentence_transformers import SentenceTransformer

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

def read_data(path, country):
    corpus = []
    limit = 0
    if "corpus" in path:
        with open(path, 'r', encoding='utf-8') as f:
            parser = ijson.parse(f)
            aux = {"country": "", "label": "", "text": ""}
            for prefix, event, value in tqdm(parser):
                prefix = ".".join(prefix.split(".")[1:])
                if (prefix, event) == ("labels.item", "string"):
                    aux["label"] = value
                elif (prefix, event) == ("text", "string"):
                    aux["text"] = value
                elif (prefix, event) == ("country", "string"):
                    aux["country"] = value

                if aux["country"] == country and aux["text"] != "" and aux["label"] != "":
                    corpus.append(aux)
                    aux = {"country": "", "label": "", "text": ""}
                    limit += 1
    else:
        with open(path, 'r', encoding='utf-8') as f:
            parser = ijson.parse(f)
            aux = {"country": "", "label": "", "text": ""}
            for prefix, event, value in parser:
                prefix = ".".join(prefix.split(".")[1:])
                if (prefix, event) == ("label", "string"):
                    aux["label"] = value
                elif (prefix, event) == ("text", "string"):
                    aux["text"] = value
                elif (prefix, event) == ("country", "string"):
                    aux["country"] = value

                if aux["country"] == country and aux["text"] != "" and aux["label"] != "":
                    corpus.append(aux)
                    aux = {"country": "", "label": "", "text": ""}
                    limit += 1

    return corpus

cuda


In [11]:
models = ["distiluse-base-multilingual-cased-v2", "paraphrase-multilingual-mpnet-base-v2"]
model_map = {"distiluse-base-multilingual-cased-v2": "distiluse", "paraphrase-multilingual-mpnet-base-v2": "mpnet"}

for model_name in models:
    model = SentenceTransformer("../models/" + model_name).to(device)

    data = read_data("data/ML-dataset/corpus.json", "ITA")
    data = pd.DataFrame(data)
    data["text"] = data["text"].apply(model.encode)

    data.to_pickle(f"embeddings/{model_map[model_name]}_corpus.pickle")

112897835it [01:01, 1845314.30it/s]
112897835it [01:01, 1832561.23it/s]
